In [61]:
!pip install kaggle

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [62]:
# download dataset and extract it
! export KAGGLE_USERNAME="rahulsaraf" && export KAGGLE_KEY="32d91d33d6411ddb24234f0965a72af7" && kaggle datasets download --unzip rahulsaraf/airpollutiondataset
! ls

  0%|                                               | 0.00/4.70k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 4.70k/4.70k [00:00<00:00, 3.51MB/s]
Data.csv  DataUpdated.csv


In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import itertools
warnings.filterwarnings("ignore")
import statsmodels.api as sm
import statistics as st
from pylab import rcParams
pd.set_option('display.max_columns', 500)
from datetime import datetime as dt
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse, aic
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from scipy import stats
from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


def pe(actual, pred): # Percentage Error
    actual, pred = np.array(actual), np.array(pred)
    return ((actual - pred) / (actual)) * 100

def ae(actual, pred): # Absolute Error
    actual, pred = np.array(actual), np.array(pred)
    return np.abs((actual - pred))

def mape(actual, pred): # Mean Absolute Percentage Error
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / (actual))) * 100


In [64]:
data = pd.read_csv("/home/wsuser/work/DataUpdated.csv")

In [65]:
data.head()

,year,co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire
0,1990,578.687,0.663,0.702,2.550,0.254,0.026,0.420,0.024,0.181,1.355,0.605,0.162,2607.675,873278016.0,639390.0,273.046613,1275490,NaN,NaN,NaN,NaN,NaN
1,1991,616.106,0.691,0.714,2.659,0.257,0.027,0.442,0.026,0.187,1.389,0.599,0.164,2694.012,891273024.0,640841.0,291.953805,1386505,NaN,NaN,NaN,NaN,NaN
2,1992,656.223,0.722,0.756,2.924,0.260,0.028,0.453,0.030,0.207,1.396,0.593,0.165,2778.159,909307008.0,642292.0,305.535929,1557013,NaN,NaN,NaN,NaN,NaN
3,1993,678.116,0.731,0.749,2.990,0.262,0.029,0.468,0.029,0.202,1.403,0.587,0.165,2792.053,927404032.0,643743.0,321.711471,1847220,NaN,NaN,NaN,NaN,NaN
4,1994,717.127,0.758,0.772,3.139,0.263,0.030,0.483,0.031,0.210,1.425,0.581,0.168,2883.378,945601984.0,645194.0,342.461839,2373632,NaN,NaN,NaN,NaN,NaN


In [66]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 23 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   year                                       31 non-null     int64  
 1   co2                                        30 non-null     float64
 2   co2_per_capita                             30 non-null     float64
 3   consumption_co2_per_capita                 29 non-null     float64
 4   share_global_co2                           30 non-null     float64
 5   co2_per_unit_energy                        27 non-null     float64
 6   cement_co2_per_capita                      30 non-null     float64
 7   coal_co2_per_capita                        30 non-null     float64
 8   gas_co2_per_capita                         30 non-null     float64
 9   oil_co2_per_capita                         30 non-null     float64
 10  ghg_per_capita              

In [67]:
data.rename(columns = {'forest_area(sq km)':'forest_area_sq_km'}, inplace = True)
data.rename(columns = {'electric_power_consumption (kWh per capita)':'electric_power_consumption_kWh_per_capita'}, inplace = True)

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 23 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   year                                       31 non-null     int64  
 1   co2                                        30 non-null     float64
 2   co2_per_capita                             30 non-null     float64
 3   consumption_co2_per_capita                 29 non-null     float64
 4   share_global_co2                           30 non-null     float64
 5   co2_per_unit_energy                        27 non-null     float64
 6   cement_co2_per_capita                      30 non-null     float64
 7   coal_co2_per_capita                        30 non-null     float64
 8   gas_co2_per_capita                         30 non-null     float64
 9   oil_co2_per_capita                         30 non-null     float64
 10  ghg_per_capita              

In [69]:
print(data.isnull().sum())

year                                          0
co2                                           1
co2_per_capita                                1
consumption_co2_per_capita                    2
share_global_co2                              1
co2_per_unit_energy                           4
cement_co2_per_capita                         1
coal_co2_per_capita                           1
gas_co2_per_capita                            1
oil_co2_per_capita                            1
ghg_per_capita                                4
methane_per_capita                            4
nitrous_oxide_per_capita                      4
energy_per_capita                             4
population                                    1
forest_area_sq_km                             5
electric_power_consumption_kWh_per_capita     6
vehicle_sales                                 0
commodity_driven_deforestation               11
forestry                                     11
shifting_agriculture                    

## DATA IMPUTATION

In [70]:
def imputeWithPercentIncrease(df, index):
    
    i = index-1
    a = df[i]
    j = index-2
    b= df[j]
    percentIncrease = ((a-b)/b)+1
    
    return a*(percentIncrease)

def imputeWithpreviousAvg(df, index):
    
    i = index-1
    a = df[i]
    j = index-2
    b= df[j]
    
    return 0.4*(a+b)

def imputeData(data):
    
    temp1 = data.iloc[:,:18]
    temp2 = data[::-1].reset_index(drop=True).iloc[:,18:]
    
    for x in temp1.columns:
        for i in range(0,31):
                if(pd.isnull(temp1[x][i])):
                    temp1[x][i]=imputeWithPercentIncrease(temp1[x],i)
                    
    for y in temp2.columns:
        for i in range(0,31):
                if(pd.isnull(temp2[y][i])):
                    temp2[y][i]=imputeWithpreviousAvg(temp2[y],i)

    temp2 = temp2[::-1].reset_index(drop=True)

    return pd.concat([temp1,temp2], axis=1)

def calculate_per_capita(col):
    temp = col.copy()
    for x in range(0,31):
        temp[x] = col[x]/data.population[x]
    return temp

In [71]:
data = imputeData(data)
data.isnull().sum()

year                                         0
co2                                          0
co2_per_capita                               0
consumption_co2_per_capita                   0
share_global_co2                             0
co2_per_unit_energy                          0
cement_co2_per_capita                        0
coal_co2_per_capita                          0
gas_co2_per_capita                           0
oil_co2_per_capita                           0
ghg_per_capita                               0
methane_per_capita                           0
nitrous_oxide_per_capita                     0
energy_per_capita                            0
population                                   0
forest_area_sq_km                            0
electric_power_consumption_kWh_per_capita    0
vehicle_sales                                0
commodity_driven_deforestation               0
forestry                                     0
shifting_agriculture                         0
urbanization 

In [72]:
data['commodity_driven_deforestation_per_capita'] = calculate_per_capita(data['commodity_driven_deforestation'])
data['forestry_per_capita'] = calculate_per_capita(data['forestry'])
data['shifting_agriculture_per_capita'] = calculate_per_capita(data['shifting_agriculture'])
data['urbanization_per_capita'] = calculate_per_capita(data['urbanization'])
data['wildfire_per_capita'] = calculate_per_capita(data['wildfire'])

In [73]:
data

,year,co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita
0,1990,578.687000,0.663000,0.702000,2.550000,0.254000,0.026000,0.420000,0.024000,0.181000,1.355000,0.605,0.162000,2607.675000,8.732780e+08,639390.000000,273.046613,1275490,1.092173e+05,4.376517e+06,6.665529e+05,7885.698839,11939.441176,0.000125,0.005012,0.000763,0.000009,0.000014
1,1991,616.106000,0.691000,0.714000,2.659000,0.257000,0.027000,0.442000,0.026000,0.187000,1.389000,0.599,0.164000,2694.012000,8.912730e+08,640841.000000,291.953805,1386505,1.264775e+05,5.071185e+06,7.722244e+05,9138.339884,13835.420387,0.000142,0.005690,0.000866,0.000010,0.000016
2,1992,656.223000,0.722000,0.756000,2.924000,0.260000,0.028000,0.453000,0.030000,0.207000,1.396000,0.593,0.165000,2778.159000,9.093070e+08,642292.000000,305.535929,1557013,1.465658e+05,5.870107e+06,8.941578e+05,10575.907215,16013.182554,0.000161,0.006456,0.000983,0.000012,0.000018
3,1993,678.116000,0.731000,0.749000,2.990000,0.262000,0.029000,0.468000,0.029000,0.202000,1.403000,0.587,0.165000,2792.053000,9.274040e+08,643743.000000,321.711471,1847220,1.696280e+05,6.807857e+06,1.036403e+06,12269.942496,18575.368414,0.000183,0.007341,0.001118,0.000013,0.000020
4,1994,717.127000,0.758000,0.772000,3.139000,0.263000,0.030000,0.483000,0.031000,0.210000,1.425000,0.581,0.168000,2883.378000,9.456020e+08,645194.000000,342.461839,2373632,1.967866e+05,7.867411e+06,1.198991e+06,14169.825541,21457.587970,0.000208,0.008320,0.001268,0.000015,0.000023
5,1995,762.338000,0.791000,0.794000,3.267000,0.260000,0.032000,0.490000,0.041000,0.224000,1.467000,0.578,0.170000,3042.622000,9.639230e+08,646645.000000,360.047053,2857013,2.272833e+05,9.152231e+06,1.392017e+06,16505.030698,24980.833065,0.000236,0.009495,0.001444,0.000017,0.000026
6,1996,826.151000,0.841000,0.832000,3.435000,0.271000,0.035000,0.515000,0.040000,0.248000,1.485000,0.572,0.171000,3107.973000,9.823650e+08,648096.000000,361.092501,3500581,2.646833e+05,1.051630e+07,1.605462e+06,18919.533155,28663.136860,0.000269,0.010705,0.001634,0.000019,0.000029
7,1997,859.929000,0.859000,0.853000,3.555000,0.267000,0.037000,0.528000,0.046000,0.244000,1.514000,0.568,0.174000,3220.282000,1.000900e+09,649547.000000,376.800678,3965383,3.035248e+05,1.236428e+07,1.874580e+06,22343.043590,33788.945804,0.000303,0.012353,0.001873,0.000022,0.000034
8,1998,877.939000,0.861000,0.854000,3.641000,0.257000,0.038000,0.515000,0.047000,0.258000,1.518000,0.567,0.176000,3350.664000,1.019484e+09,650998.000000,387.196900,3985508,3.581835e+05,1.392646e+07,2.139074e+06,24955.789296,37868.896346,0.000351,0.013660,0.002098,0.000024,0.000037
9,1999,951.671000,0.917000,0.922000,3.895000,0.271000,0.043000,0.541000,0.041000,0.289000,1.567000,0.565,0.177000,3379.954000,1.038058e+09,652449.000000,393.373199,4274888,4.006285e+05,1.698424e+07,2.547375e+06,30901.819680,46603.468164,0.000386,0.016362,0.002454,0.000030,0.000045


In [74]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['population_scaled'] = scaler.fit_transform(data.population.values.reshape(-1, 1))
data['forest_area_scaled'] = scaler.fit_transform(data.forest_area_sq_km.values.reshape(-1, 1))
data['vehicle_sales_scaled'] = scaler.fit_transform(data.vehicle_sales.values.reshape(-1, 1))
data.head()

,year,co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita,population_scaled,forest_area_scaled,vehicle_sales_scaled
0,1990,578.687,0.663,0.702,2.550,0.254,0.026,0.420,0.024,0.181,1.355,0.605,0.162,2607.675,873278016.0,639390.0,273.046613,1275490,109217.348909,4.376517e+06,6.665529e+05,7885.698839,11939.441176,0.000125,0.005012,0.000763,0.000009,0.000014,0.000000,0.000000,0.000000
1,1991,616.106,0.691,0.714,2.659,0.257,0.027,0.442,0.026,0.187,1.389,0.599,0.164,2694.012,891273024.0,640841.0,291.953805,1386505,126477.525774,5.071185e+06,7.722244e+05,9138.339884,13835.420387,0.000142,0.005690,0.000866,0.000010,0.000016,0.035489,0.018988,0.004442
2,1992,656.223,0.722,0.756,2.924,0.260,0.028,0.453,0.030,0.207,1.396,0.593,0.165,2778.159,909307008.0,642292.0,305.535929,1557013,146565.846499,5.870107e+06,8.941578e+05,10575.907215,16013.182554,0.000161,0.006456,0.000983,0.000012,0.000018,0.071055,0.037975,0.011265
3,1993,678.116,0.731,0.749,2.990,0.262,0.029,0.468,0.029,0.202,1.403,0.587,0.165,2792.053,927404032.0,643743.0,321.711471,1847220,169627.967935,6.807857e+06,1.036403e+06,12269.942496,18575.368414,0.000183,0.007341,0.001118,0.000013,0.000020,0.106746,0.056963,0.022878
4,1994,717.127,0.758,0.772,3.139,0.263,0.030,0.483,0.031,0.210,1.425,0.581,0.168,2883.378,945601984.0,645194.0,342.461839,2373632,196786.648311,7.867411e+06,1.198991e+06,14169.825541,21457.587970,0.000208,0.008320,0.001268,0.000015,0.000023,0.142635,0.075951,0.043942


In [75]:
data.year = pd.to_datetime(data.year, format='%Y')

In [76]:
data = data.set_index('year')

In [77]:
data.head()

,co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita,population_scaled,forest_area_scaled,vehicle_sales_scaled
year,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1990-01-01,578.687,0.663,0.702,2.550,0.254,0.026,0.420,0.024,0.181,1.355,0.605,0.162,2607.675,873278016.0,639390.0,273.046613,1275490,109217.348909,4.376517e+06,6.665529e+05,7885.698839,11939.441176,0.000125,0.005012,0.000763,0.000009,0.000014,0.000000,0.000000,0.000000
1991-01-01,616.106,0.691,0.714,2.659,0.257,0.027,0.442,0.026,0.187,1.389,0.599,0.164,2694.012,891273024.0,640841.0,291.953805,1386505,126477.525774,5.071185e+06,7.722244e+05,9138.339884,13835.420387,0.000142,0.005690,0.000866,0.000010,0.000016,0.035489,0.018988,0.004442
1992-01-01,656.223,0.722,0.756,2.924,0.260,0.028,0.453,0.030,0.207,1.396,0.593,0.165,2778.159,909307008.0,642292.0,305.535929,1557013,146565.846499,5.870107e+06,8.941578e+05,10575.907215,16013.182554,0.000161,0.006456,0.000983,0.000012,0.000018,0.071055,0.037975,0.011265
1993-01-01,678.116,0.731,0.749,2.990,0.262,0.029,0.468,0.029,0.202,1.403,0.587,0.165,2792.053,927404032.0,643743.0,321.711471,1847220,169627.967935,6.807857e+06,1.036403e+06,12269.942496,18575.368414,0.000183,0.007341,0.001118,0.000013,0.000020,0.106746,0.056963,0.022878
1994-01-01,717.127,0.758,0.772,3.139,0.263,0.030,0.483,0.031,0.210,1.425,0.581,0.168,2883.378,945601984.0,645194.0,342.461839,2373632,196786.648311,7.867411e+06,1.198991e+06,14169.825541,21457.587970,0.000208,0.008320,0.001268,0.000015,0.000023,0.142635,0.075951,0.043942


In [78]:
data.columns

Index(['co2', 'co2_per_capita', 'consumption_co2_per_capita',
       'share_global_co2', 'co2_per_unit_energy', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'ghg_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita',
       'energy_per_capita', 'population', 'forest_area_sq_km',
       'electric_power_consumption_kWh_per_capita', 'vehicle_sales',
       'commodity_driven_deforestation', 'forestry', 'shifting_agriculture',
       'urbanization', 'wildfire', 'commodity_driven_deforestation_per_capita',
       'forestry_per_capita', 'shifting_agriculture_per_capita',
       'urbanization_per_capita', 'wildfire_per_capita', 'population_scaled',
       'forest_area_scaled', 'vehicle_sales_scaled'],
      dtype='object')

In [79]:
grp_columns =['co2', 'consumption_co2_per_capita',
       'share_global_co2', 'co2_per_unit_energy', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'ghg_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita',
       'energy_per_capita', 'population', 'forest_area_sq_km',
       'electric_power_consumption_kWh_per_capita', 'vehicle_sales',
       'commodity_driven_deforestation', 'forestry', 'shifting_agriculture',
       'urbanization', 'wildfire', 'commodity_driven_deforestation_per_capita',
       'forestry_per_capita', 'shifting_agriculture_per_capita',
       'urbanization_per_capita', 'wildfire_per_capita', 'population_scaled',
       'forest_area_scaled', 'vehicle_sales_scaled']

In [80]:
!pip install pmdarima

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [81]:
from pmdarima.arima import auto_arima

In [82]:
result = pd.DataFrame()
testing = pd.DataFrame()

In [86]:
for i in grp_columns:
    train = data[i][:'2016-01-01'] # dataset to train
    test = data[i]['2017-01-01':] # last X months for test
    arima_model =  auto_arima(train,start_p=0, d=1, start_q=0, 
                          max_p=5, max_d=5, max_q=5, start_P=0, 
                          D=1, start_Q=0, max_P=5, max_D=5,
                          max_Q=5, m=3, seasonal=False, 
                          error_action='warn',trace = True,
                          supress_warnings=True,stepwise = True,
                          random_state=20,n_fits = 50 )
    prediction = pd.DataFrame(arima_model.predict(n_periods = 4),index=test.index)
    prediction.columns = ['test_pred']
    mape=mean_absolute_percentage_error(y_true=test,y_pred=prediction)
    print(mape)
    arma_df = arima_model.predict(start='2021-01-01', end='2030-01-01')
    testing[i]=arma_df
    result = pd.concat([testing], axis=1)

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=274.034, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=269.706, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=272.924, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=305.085, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=264.623, Time=0.08 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=265.143, Time=0.10 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=265.111, Time=0.12 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=265.438, Time=0.11 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=266.911, Time=0.16 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=inf, Time=0.07 sec

Best model:  ARIMA(2,1,0)(0,0,0)[0] intercept
Total fit time: 0.788 seconds
4.4203931058397785
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-100.690, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-98.861, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC

 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=464.033, Time=0.08 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.256 seconds
0.4812012035808689
Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=218.281, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=204.303, Time=0.11 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=211.348, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=249.227, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=204.184, Time=0.06 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=205.010, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=204.941, Time=0.09 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=202.979, Time=0.06 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=204.901, Time=0.09 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=208.769, Time=0.07 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=203.450, Time=0.04 sec

Best model:  ARIMA(

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-229.643, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=-300.912, Time=0.20 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.12 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=-99.539, Time=0.01 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=-280.262, Time=0.25 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=-335.277, Time=0.29 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=-305.037, Time=0.27 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=-298.678, Time=0.34 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-283.702, Time=0.26 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=-381.437, Time=0.11 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=-256.975, Time=0.26 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=-379.714, Time=0.30 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=-362.858, Time=0.45 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=-284.131, Time=0.38 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : A

In [87]:
result

,co2,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita,population_scaled,forest_area_scaled,vehicle_sales_scaled
0,2477.903683,1.682308,6.956192,0.285500,0.095679,1.189394,0.087800,0.472808,2.484846,0.497000,0.187961,6456.526145,1.341872e+09,711270.764720,932.177762,2.215293e+07,3.672710e+06,6.420610e+07,6.373770e+06,54711.194158,127482.731,0.002748,0.048758,0.004879,0.000079,0.000099,0.918010,0.930499,0.837125
1,2586.554716,1.718615,7.119384,0.286618,0.098267,1.219751,0.088611,0.483615,2.526692,0.493001,0.188922,6624.774930,1.359228e+09,713933.015266,971.179916,2.275783e+07,3.914228e+06,7.591667e+07,4.572612e+06,98007.293046,127482.731,0.002946,0.057217,0.005028,0.000081,0.000103,0.945990,0.956293,0.867476
2,2673.513584,1.754923,7.282576,0.287736,0.100830,1.247639,0.088975,0.494423,2.568538,0.489001,0.189884,6768.132546,1.376583e+09,716595.265811,1009.157854,2.336274e+07,4.014891e+06,7.279440e+07,6.939648e+06,66564.475746,127482.731,0.003007,0.055028,0.005177,0.000084,0.000106,0.973979,0.982987,0.897828
3,2767.177122,1.791231,7.445768,0.288853,0.103385,1.276236,0.089139,0.505231,2.610384,0.485002,0.190845,6923.213087,1.393938e+09,719257.516356,1046.195192,2.396764e+07,4.165966e+06,7.864194e+07,5.187138e+06,97587.536860,127482.731,0.003124,0.059189,0.005326,0.000086,0.000109,1.002095,1.010427,0.928180
4,2851.410458,1.827538,7.608960,0.289971,0.105938,1.304331,0.089213,0.516038,2.652230,0.481002,0.191806,7066.016788,1.411294e+09,721919.766901,1082.368719,2.457255e+07,4.298999e+06,7.906519e+07,5.736933e+06,76402.375471,127482.731,0.003218,0.059562,0.005475,0.000089,0.000112,1.030445,1.038486,0.958532
5,2936.869103,1.863846,7.772152,0.291089,0.108491,1.332483,0.089246,0.526846,2.694077,0.477003,0.192767,7213.668207,1.428649e+09,724582.017446,1117.748955,2.517745e+07,4.438488e+06,8.276866e+07,6.295134e+06,98852.212358,127482.731,0.003321,0.062195,0.005624,0.000092,0.000115,1.059125,1.067059,0.988883
6,3017.840278,1.900154,7.935344,0.292206,0.111043,1.360508,0.089261,0.537654,2.735923,0.473003,0.193729,7355.135229,1.446004e+09,727244.267992,1152.400663,2.578236e+07,4.575667e+06,8.448850e+07,5.176766e+06,84832.447652,127482.731,0.003421,0.063480,0.005773,0.000094,0.000118,1.088219,1.096058,1.019235
7,3098.400747,1.936462,8.098536,0.293324,0.113595,1.388513,0.089268,0.548462,2.777769,0.469004,0.194690,7498.453726,1.463360e+09,729906.518537,1186.383320,2.638726e+07,4.713673e+06,8.740789e+07,6.216628e+06,101293.533019,127482.731,0.003522,0.065569,0.005922,0.000097,0.000122,1.117796,1.125412,1.049587
8,3176.597625,1.972769,8.261728,0.294442,0.116147,1.416476,0.089271,0.559269,2.819615,0.465004,0.195651,7638.575197,1.480715e+09,732568.769082,1219.751546,2.699216e+07,4.851383e+06,8.960187e+07,5.683392e+06,92279.094098,127482.731,0.003622,0.067179,0.006071,0.000099,0.000125,1.147914,1.155059,1.079939
9,3254.044122,2.009077,8.424919,0.295560,0.118699,1.444420,0.089273,0.570077,2.861461,0.461004,0.196612,7779.288316,1.498070e+09,735231.019627,1252.555503,2.759707e+07,4.989198e+06,9.223453e+07,5.619415e+06,104556.788965,127482.731,0.003723,0.069075,0.006220,0.000102,0.000128,1.178613,1.184950,1.110290


In [88]:
dates = pd.DataFrame({'date':[2021,2022,2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]})

In [89]:
testesting_data= pd.concat([dates, result], axis=1)

In [90]:
testesting_data

,date,co2,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita,population_scaled,forest_area_scaled,vehicle_sales_scaled
0,2021,2477.903683,1.682308,6.956192,0.285500,0.095679,1.189394,0.087800,0.472808,2.484846,0.497000,0.187961,6456.526145,1.341872e+09,711270.764720,932.177762,2.215293e+07,3.672710e+06,6.420610e+07,6.373770e+06,54711.194158,127482.731,0.002748,0.048758,0.004879,0.000079,0.000099,0.918010,0.930499,0.837125
1,2022,2586.554716,1.718615,7.119384,0.286618,0.098267,1.219751,0.088611,0.483615,2.526692,0.493001,0.188922,6624.774930,1.359228e+09,713933.015266,971.179916,2.275783e+07,3.914228e+06,7.591667e+07,4.572612e+06,98007.293046,127482.731,0.002946,0.057217,0.005028,0.000081,0.000103,0.945990,0.956293,0.867476
2,2023,2673.513584,1.754923,7.282576,0.287736,0.100830,1.247639,0.088975,0.494423,2.568538,0.489001,0.189884,6768.132546,1.376583e+09,716595.265811,1009.157854,2.336274e+07,4.014891e+06,7.279440e+07,6.939648e+06,66564.475746,127482.731,0.003007,0.055028,0.005177,0.000084,0.000106,0.973979,0.982987,0.897828
3,2024,2767.177122,1.791231,7.445768,0.288853,0.103385,1.276236,0.089139,0.505231,2.610384,0.485002,0.190845,6923.213087,1.393938e+09,719257.516356,1046.195192,2.396764e+07,4.165966e+06,7.864194e+07,5.187138e+06,97587.536860,127482.731,0.003124,0.059189,0.005326,0.000086,0.000109,1.002095,1.010427,0.928180
4,2025,2851.410458,1.827538,7.608960,0.289971,0.105938,1.304331,0.089213,0.516038,2.652230,0.481002,0.191806,7066.016788,1.411294e+09,721919.766901,1082.368719,2.457255e+07,4.298999e+06,7.906519e+07,5.736933e+06,76402.375471,127482.731,0.003218,0.059562,0.005475,0.000089,0.000112,1.030445,1.038486,0.958532
5,2026,2936.869103,1.863846,7.772152,0.291089,0.108491,1.332483,0.089246,0.526846,2.694077,0.477003,0.192767,7213.668207,1.428649e+09,724582.017446,1117.748955,2.517745e+07,4.438488e+06,8.276866e+07,6.295134e+06,98852.212358,127482.731,0.003321,0.062195,0.005624,0.000092,0.000115,1.059125,1.067059,0.988883
6,2027,3017.840278,1.900154,7.935344,0.292206,0.111043,1.360508,0.089261,0.537654,2.735923,0.473003,0.193729,7355.135229,1.446004e+09,727244.267992,1152.400663,2.578236e+07,4.575667e+06,8.448850e+07,5.176766e+06,84832.447652,127482.731,0.003421,0.063480,0.005773,0.000094,0.000118,1.088219,1.096058,1.019235
7,2028,3098.400747,1.936462,8.098536,0.293324,0.113595,1.388513,0.089268,0.548462,2.777769,0.469004,0.194690,7498.453726,1.463360e+09,729906.518537,1186.383320,2.638726e+07,4.713673e+06,8.740789e+07,6.216628e+06,101293.533019,127482.731,0.003522,0.065569,0.005922,0.000097,0.000122,1.117796,1.125412,1.049587
8,2029,3176.597625,1.972769,8.261728,0.294442,0.116147,1.416476,0.089271,0.559269,2.819615,0.465004,0.195651,7638.575197,1.480715e+09,732568.769082,1219.751546,2.699216e+07,4.851383e+06,8.960187e+07,5.683392e+06,92279.094098,127482.731,0.003622,0.067179,0.006071,0.000099,0.000125,1.147914,1.155059,1.079939
9,2030,3254.044122,2.009077,8.424919,0.295560,0.118699,1.444420,0.089273,0.570077,2.861461,0.461004,0.196612,7779.288316,1.498070e+09,735231.019627,1252.555503,2.759707e+07,4.989198e+06,9.223453e+07,5.619415e+06,104556.788965,127482.731,0.003723,0.069075,0.006220,0.000102,0.000128,1.178613,1.184950,1.110290


In [91]:
testesting_data.date = pd.to_datetime(testesting_data.date, format='%Y')

In [92]:
testesting_data = testesting_data.set_index('date')

In [93]:
testesting_data

,co2,consumption_co2_per_capita,share_global_co2,co2_per_unit_energy,cement_co2_per_capita,coal_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,ghg_per_capita,methane_per_capita,nitrous_oxide_per_capita,energy_per_capita,population,forest_area_sq_km,electric_power_consumption_kWh_per_capita,vehicle_sales,commodity_driven_deforestation,forestry,shifting_agriculture,urbanization,wildfire,commodity_driven_deforestation_per_capita,forestry_per_capita,shifting_agriculture_per_capita,urbanization_per_capita,wildfire_per_capita,population_scaled,forest_area_scaled,vehicle_sales_scaled
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-01-01,2477.903683,1.682308,6.956192,0.285500,0.095679,1.189394,0.087800,0.472808,2.484846,0.497000,0.187961,6456.526145,1.341872e+09,711270.764720,932.177762,2.215293e+07,3.672710e+06,6.420610e+07,6.373770e+06,54711.194158,127482.731,0.002748,0.048758,0.004879,0.000079,0.000099,0.918010,0.930499,0.837125
2022-01-01,2586.554716,1.718615,7.119384,0.286618,0.098267,1.219751,0.088611,0.483615,2.526692,0.493001,0.188922,6624.774930,1.359228e+09,713933.015266,971.179916,2.275783e+07,3.914228e+06,7.591667e+07,4.572612e+06,98007.293046,127482.731,0.002946,0.057217,0.005028,0.000081,0.000103,0.945990,0.956293,0.867476
2023-01-01,2673.513584,1.754923,7.282576,0.287736,0.100830,1.247639,0.088975,0.494423,2.568538,0.489001,0.189884,6768.132546,1.376583e+09,716595.265811,1009.157854,2.336274e+07,4.014891e+06,7.279440e+07,6.939648e+06,66564.475746,127482.731,0.003007,0.055028,0.005177,0.000084,0.000106,0.973979,0.982987,0.897828
2024-01-01,2767.177122,1.791231,7.445768,0.288853,0.103385,1.276236,0.089139,0.505231,2.610384,0.485002,0.190845,6923.213087,1.393938e+09,719257.516356,1046.195192,2.396764e+07,4.165966e+06,7.864194e+07,5.187138e+06,97587.536860,127482.731,0.003124,0.059189,0.005326,0.000086,0.000109,1.002095,1.010427,0.928180
2025-01-01,2851.410458,1.827538,7.608960,0.289971,0.105938,1.304331,0.089213,0.516038,2.652230,0.481002,0.191806,7066.016788,1.411294e+09,721919.766901,1082.368719,2.457255e+07,4.298999e+06,7.906519e+07,5.736933e+06,76402.375471,127482.731,0.003218,0.059562,0.005475,0.000089,0.000112,1.030445,1.038486,0.958532
2026-01-01,2936.869103,1.863846,7.772152,0.291089,0.108491,1.332483,0.089246,0.526846,2.694077,0.477003,0.192767,7213.668207,1.428649e+09,724582.017446,1117.748955,2.517745e+07,4.438488e+06,8.276866e+07,6.295134e+06,98852.212358,127482.731,0.003321,0.062195,0.005624,0.000092,0.000115,1.059125,1.067059,0.988883
2027-01-01,3017.840278,1.900154,7.935344,0.292206,0.111043,1.360508,0.089261,0.537654,2.735923,0.473003,0.193729,7355.135229,1.446004e+09,727244.267992,1152.400663,2.578236e+07,4.575667e+06,8.448850e+07,5.176766e+06,84832.447652,127482.731,0.003421,0.063480,0.005773,0.000094,0.000118,1.088219,1.096058,1.019235
2028-01-01,3098.400747,1.936462,8.098536,0.293324,0.113595,1.388513,0.089268,0.548462,2.777769,0.469004,0.194690,7498.453726,1.463360e+09,729906.518537,1186.383320,2.638726e+07,4.713673e+06,8.740789e+07,6.216628e+06,101293.533019,127482.731,0.003522,0.065569,0.005922,0.000097,0.000122,1.117796,1.125412,1.049587
2029-01-01,3176.597625,1.972769,8.261728,0.294442,0.116147,1.416476,0.089271,0.559269,2.819615,0.465004,0.195651,7638.575197,1.480715e+09,732568.769082,1219.751546,2.699216e+07,4.851383e+06,8.960187e+07,5.683392e+06,92279.094098,127482.731,0.003622,0.067179,0.006071,0.000099,0.000125,1.147914,1.155059,1.079939


In [94]:
a_to_train_3 = data[:'2016-01-01'] # dataset to train
a_to_val_3 = data['2017-01-01':] # last X months for test  
predict_date = len(data) - len(data[:'2016-01-01'])
X_train3 = a_to_train_3[['co2', 'consumption_co2_per_capita',
       'share_global_co2', 'co2_per_unit_energy', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'ghg_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita',
       'energy_per_capita', 'population', 'forest_area_sq_km',
       'electric_power_consumption_kWh_per_capita', 'vehicle_sales',
       'commodity_driven_deforestation', 'forestry', 'shifting_agriculture',
       'urbanization', 'wildfire', 'commodity_driven_deforestation_per_capita',
       'forestry_per_capita', 'shifting_agriculture_per_capita',
       'urbanization_per_capita', 'wildfire_per_capita', 'population_scaled',
       'forest_area_scaled', 'vehicle_sales_scaled']]
y_train3 = a_to_train_3['co2_per_capita']
X_test3 = a_to_val_3[['co2', 'consumption_co2_per_capita',
       'share_global_co2', 'co2_per_unit_energy', 'cement_co2_per_capita',
       'coal_co2_per_capita', 'gas_co2_per_capita', 'oil_co2_per_capita',
       'ghg_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita',
       'energy_per_capita', 'population', 'forest_area_sq_km',
       'electric_power_consumption_kWh_per_capita', 'vehicle_sales',
       'commodity_driven_deforestation', 'forestry', 'shifting_agriculture',
       'urbanization', 'wildfire', 'commodity_driven_deforestation_per_capita',
       'forestry_per_capita', 'shifting_agriculture_per_capita',
       'urbanization_per_capita', 'wildfire_per_capita', 'population_scaled',
       'forest_area_scaled', 'vehicle_sales_scaled']]
y_test3 = a_to_val_3['co2_per_capita']

In [122]:
param_grid = { 'n_estimators': [200, 500],'max_features': ['auto', 'sqrt', 'log2'],'max_depth' : [1,4,5]}
rfc=RandomForestRegressor(random_state=42)
grid = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5, n_jobs = 3)
grid.fit(X_train3, y_train3)
predw1 = grid.best_estimator_.predict(X_test3)

In [126]:
measure_2 = pd.DataFrame(columns = ['Actual', 'Predicted','Error','PE'])
measure_2['Actual'] =a_to_val_3['co2_per_capita']
measure_2['Predicted'] = predw1
measure_2['Error'] = measure_2['Actual'] - measure_2['Predicted'] 
measure_2['PE'] = pe(measure_2['Actual'],measure_2['Predicted'])

In [127]:
measure_2

,Actual,Predicted,Error,PE
year,,,,
2017-01-01,1.835000,1.727604,0.107396,5.852643
2018-01-01,1.916000,1.709350,0.206650,10.785491
2019-01-01,1.915000,1.694368,0.220632,11.521253
2020-01-01,1.914001,1.692208,0.221793,11.587903


In [128]:
mae_rf=mean_absolute_error(y_true=measure_2['Actual'],y_pred=measure_2['Predicted'])
mape_rf =mean_absolute_percentage_error(y_true=measure_2['Actual'],y_pred=measure_2['Predicted'])

In [129]:
mape_rf

9.936822454377175

In [130]:
mae_rf

0.18911763048015595

In [123]:
future_valuesrf = grid.best_estimator_.predict(testesting_data)

In [124]:
future_valuesrf

array([1.7455  , 1.750174, 1.746706, 1.748038, 1.743278, 1.74201 ,
       1.735572, 1.737008, 1.73548 , 1.736222])

In [131]:
future_valuesrf= pd.DataFrame(future_valuesrf)

In [132]:
future_valuesrf.columns =['co2_per_capita']

In [133]:
dates = [2021, 2022, 2023, 2024, 2025, 2026, 2027, 2028, 2029, 2030]

In [134]:
future_valuesrf['year']= dates

In [135]:
future_valuesrf

,co2_per_capita,year
0,1.745500,2021
1,1.750174,2022
2,1.746706,2023
3,1.748038,2024
4,1.743278,2025
5,1.742010,2026
6,1.735572,2027
7,1.737008,2028
8,1.735480,2029
9,1.736222,2030


In [136]:
future_valuesrf.year = pd.to_datetime(future_valuesrf.year, format='%Y')

future_valuesrf = future_valuesrf.set_index('year')

In [137]:
future_valuesrf

,co2_per_capita
year,
2021-01-01,1.745500
2022-01-01,1.750174
2023-01-01,1.746706
2024-01-01,1.748038
2025-01-01,1.743278
2026-01-01,1.742010
2027-01-01,1.735572
2028-01-01,1.737008
2029-01-01,1.735480
